# **Data understanding**

**Data Understanding** adalah proses mendalami data: struktur, sumber, kualitas, distribusi, dan keterbatasannya. Tujuannya memastikan data layak dipakai untuk analisis & modelling.

1. Langkah-langkah Umum

* Ingest & sampling — ambil sample dataset (mis. 1k baris) untuk eksplorasi awal.

* Profiling — statistik dasar: count, unique, min/max, mean, std, percentiles.

* Visualisasi awal — hist, boxplot, scatter untuk deteksi pola & outlier.

* Quality checks — missing, duplicates, inconsistent formats, range checks.

* Data dictionary / metadata — jelaskan tiap kolom: tipe, allowed values, deskripsi.

* Lineage & source validation — darimana data? apakah ETL mengubahnya?

* Privacy & compliance — PII handling, GDPR/PDPA bila relevan.

2. Dimensi Kualitas Data (detail)

  * Completeness (kelengkapan): ada missing/null?

  * Accuracy (ketepatan): apakah nilai mewakili kenyataan? (sulit verifikasi tanpa sumber eksternal)

  * Consistency (konsistensi): format kolom konsisten (tanggal, kode pos), business rules konsisten (ShipDate >= OrderDate)

  * Uniqueness: duplicate transactions / order IDs

  * Validity: apakah nilai di domain yang valid (mis. Segment ∈ {Consumer, Corporate, Home Office})

  * Timeliness: data up-to-date sesuai kebutuhan analisis

  * Integrity / Referential: relasi antar tabel (jika ada) valid

3. Data Dictionary (contoh ringkas untuk Superstore)

  * *Order ID* — string, unik per order (mis. CA-2018-...)

  * *Order Date* — date, format (YYYY-MM-DD)

  * *Ship Date* — date

  * *Ship Mode* — categorical (First Class, Second Class, Standard Class, Same Day)

  * *Sales* — numeric (float), total sale amount

  * *Quantity* — integer

  * *Discount* — numeric (0.0 - 1.0) biasanya percent

  * *Profit* — numeric (float)
4. Praktik EDA & Pemeriksaan Kualitas

saya untuk postgrenya menggunakan database online yaitu menggunakan supabase sebelum ke power bi kita harus menambahkan data dari google colab ke supabase tersebut lalu kita lanjut ke power bi

In [ ]:
# === 1. Install library yang dibutuhkan ===
!pip install psycopg2-binary python-dotenv pandas

# === 2. Buat file .env berisi credential Supabase ===
import os

env_content = """
USER=postgres.wzdudprybyachqxjhmpw
PASSWORD=ks9unws6dHe76WJD
HOST=aws-1-us-east-2.pooler.supabase.com
PORT=5432
DBNAME=postgres
"""
with open(".env", "w") as f:
    f.write(env_content)

# === 3. Load env & koneksi ke Supabase PostgreSQL ===
import psycopg2
from dotenv import load_dotenv
import pandas as pd
from google.colab import files

load_dotenv()

USER = os.getenv("USER")
PASSWORD = os.getenv("PASSWORD")
HOST = os.getenv("HOST")
PORT = os.getenv("PORT")
DBNAME = os.getenv("DBNAME")

try:
    conn = psycopg2.connect(
        user=USER,
        password=PASSWORD,
        host=HOST,
        port=PORT,
        dbname=DBNAME
    )
    print("✅ Connection successful!")
except Exception as e:
    print("❌ Failed to connect:", e)

# === 4. Upload CSV ke Colab ===
uploaded = files.upload()   # pilih file Sample - Superstore.csv

# baca CSV dengan encoding latin1 (umum untuk file dari Excel/Windows)
df = pd.read_csv("Sample - Superstore.csv", encoding="latin1")
print("Preview data:")
print(df.head())

# === 5. Buat tabel di PostgreSQL jika belum ada ===
cursor = conn.cursor()

create_table_query = """
CREATE TABLE IF NOT EXISTS superstore (
    Row_ID INT,
    Order_ID VARCHAR(50),
    Order_Date DATE,
    Ship_Date DATE,
    Ship_Mode VARCHAR(50),
    Customer_ID VARCHAR(50),
    Customer_Name VARCHAR(100),
    Segment VARCHAR(50),
    Country VARCHAR(50),
    City VARCHAR(50),
    State VARCHAR(50),
    Postal_Code VARCHAR(20),
    Region VARCHAR(50),
    Product_ID VARCHAR(50),
    Category VARCHAR(50),
    Sub_Category VARCHAR(50),
    Product_Name VARCHAR(255),
    Sales FLOAT,
    Quantity INT,
    Discount FLOAT,
    Profit FLOAT
);
"""
cursor.execute(create_table_query)
conn.commit()
print("✅ Table ready!")

# === 6. Insert data CSV ke PostgreSQL ===
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO superstore (
            Row_ID, Order_ID, Order_Date, Ship_Date, Ship_Mode,
            Customer_ID, Customer_Name, Segment, Country, City,
            State, Postal_Code, Region, Product_ID, Category,
            Sub_Category, Product_Name, Sales, Quantity, Discount, Profit
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, tuple(row))

conn.commit()
print("✅ Data inserted to Supabase!")

# === 7. Cek apakah data berhasil masuk ===
cursor.execute("SELECT COUNT(*) FROM superstore;")
print("📊 Total rows in table:", cursor.fetchone()[0])

cursor.close()
conn.close()
print("🔒 Connection closed.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 23.6 MB/s eta 0:00:00
✅ Connection successful!


Saving Sample - Superstore.csv to Sample - Superstore.csv
Preview data:
   Row ID        Order ID  Order Date   Ship Date       Ship Mode Customer ID  \
0       1  CA-2016-152156   11/8/2016  11/11/2016    Second Class    CG-12520   
1       2  CA-2016-152156   11/8/2016  11/11/2016    Second Class    CG-12520   
2       3  CA-2016-138688   6/12/2016   6/16/2016    Second Class    DV-13045   
3       4  US-2015-108966  10/11/2015  10/18/2015  Standard Class    SO-20335   
4       5  US-2015-108966  10/11/2015  10/18/2015  Standard Class    SO-20335   

     Customer Name    Segment        Country             City  ...  \
0      Claire Gute   Consumer  United States        Henderson  ...   
1      Claire Gute   Consumer  United States        Henderson  ...   
2  Darrin Van Huff  Corporate  United States      Los Angeles  ...   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   

  Postal Code  Regio

untuk mengecek type data

In [ ]:
print(df.dtypes)


Row ID             int64
Order ID          object
Order Date        object
Ship Date         object
Ship Mode         object
Customer ID       object
Customer Name     object
Segment           object
Country           object
City              object
State             object
Postal Code        int64
Region            object
Product ID        object
Category          object
Sub-Category      object
Product Name      object
Sales            float64
Quantity           int64
Discount         float64
Profit           float64
dtype: object
